In [1]:
from selenium import webdriver


def get_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Remote(
        command_executor=f'http://localhost:4444/wd/hub',
        options=options
    )

    return driver

In [8]:
driver = get_driver()

base_url = "https://www.gov.uk/"

driver.get(base_url)

page_source = driver.page_source

with open("page_content.html", "w", encoding="utf-8") as file:
    file.write(page_source)

driver.quit()

In [10]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin


def get_section_headers(element):
    """Recursively collect text from parent section headers."""
    parent = element.find_parent()
    if parent is None:
        return []

    # Check if the parent is a header tag
    if parent.name in ["h1", "h2", "h3", "h4", "h5", "h6"]:
        header_text = parent.get_text(strip=True)
        return get_section_headers(parent) + [header_text]

    # Check if the parent is a section tag
    elif parent.name == "section":
        # Find the first header tag within the section
        header = parent.find(["h1", "h2", "h3", "h4", "h5", "h6"])
        if header:
            header_text = header.get_text(strip=True)
            return get_section_headers(parent) + [header_text]

    return get_section_headers(parent)


soup = BeautifulSoup(page_source, "html.parser")

links = soup.find_all("a")

for link in links:
    url = link.get("href")
    text = link.get_text(strip=True)
    full_url = urljoin(base_url, url)

    # parent_text_tree = get_section_headers(link)
    # print(f"URL: {url}, Text: {text}, Parent Text Tree: {parent_text_tree}")
    print(f"URL: {full_url}, Text: {text}")

URL: /help/cookies, Text: change your cookie settings, Parent Text Tree: []
URL: https://www.gov.uk/help/cookies, Text: change your cookie settings
URL: /help/cookies, Text: change your cookie settings, Parent Text Tree: []
URL: https://www.gov.uk/help/cookies, Text: change your cookie settings
URL: /help/cookies, Text: View cookies, Parent Text Tree: []
URL: https://www.gov.uk/help/cookies, Text: View cookies
URL: #content, Text: Skip to main content, Parent Text Tree: []
URL: https://www.gov.uk/#content, Text: Skip to main content
URL: https://www.gov.uk, Text: GOV.UK, Parent Text Tree: []
URL: https://www.gov.uk, Text: GOV.UK
URL: /browse, Text: Menu, Parent Text Tree: []
URL: https://www.gov.uk/browse, Text: Menu
URL: /search, Text: Search GOV.UK, Parent Text Tree: []
URL: https://www.gov.uk/search, Text: Search GOV.UK
URL: https://www.gov.uk/browse/benefits, Text: Benefits, Parent Text Tree: []
URL: https://www.gov.uk/browse/benefits, Text: Benefits
URL: https://www.gov.uk/browse/